### Column Describtions
##### CRIM - Per Capita crime rate 
##### ZN - Proportion of residential land zoned for lots over 25000 sq. ft
##### INDUS - Proportion of non-retial business acres 
##### CHAS - Charles River dummy variable (1 - if tracts bounds river, 0 -otherwise)
##### NOX - Nitrogen Oxide concentration
##### RM - Average number of rooms per dwelling
##### AGE - Proportion of owner-occupied unit built prior 1940
##### DIS - Weighted MEan of distances of five Boston Employement Centres
##### RAD - Index of accessibilities to Radial highways
##### TAX - Full-value-property-tax rates per $10,000
##### PT - Pupil-teacher Ratio
##### B - the proportion of blacks 
##### LSTAT - Lower Status of the Population (%)
##### MV - Median Value of homes (Target Variable)


In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 42.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0250ec96dda63e42b40f00dbbeba3d6ab6a4554dbb481d4c71e39d3ae047445b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
# Importing Necessary Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
appName= "hive_pyspark"
master= "local"

In [5]:
# Creting Pyspark Session
spark = SparkSession.builder \
	.master(master).appName(appName).enableHiveSupport().getOrCreate()

In [6]:
# Checking for Pre-existing Databases
df=spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



###  Read the given CSV file in a Hive table

In [7]:
# Reading the given CSV file in a Hive table
datafile=spark.read.csv("boston.csv",header=True)
datafile.show(5)
datafile.write.saveAsTable("boston_table")

+-----------+---+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM| ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+---+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632| 18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|  0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|  0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|  0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|  0|2.18000006

###  Read the data from Hive table as spark dataframe

In [20]:
# Reading the data from Hive table as spark dataframe
df=spark.sql("select * from boston_table")
df.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

### Get the correlation between dependent and independent variables


##### Methord 2

In [40]:
# Using Stat function to get Correlation
df.stat.corr('CRIM','MV')

-0.3883046116575089

In [29]:
# Printing the Data Types of the columns
df.dtypes

[('CRIM', 'double'),
 ('ZN', 'double'),
 ('INDUS', 'double'),
 ('CHAS', 'double'),
 ('NOX', 'double'),
 ('RM', 'double'),
 ('AGE', 'double'),
 ('DIS', 'double'),
 ('RAD', 'double'),
 ('TAX', 'double'),
 ('PT', 'double'),
 ('B', 'double'),
 ('LSTAT', 'double'),
 ('MV', 'double')]

In [30]:
# Printing the correlation of the IDV columns & the DV column using stst function
for col in df.dtypes:
    print('Correlation of',col[0],'&', 'MV =',df.stat.corr(col[0],'MV'))

Correlation of CRIM & MV = -0.3883046116575089
Correlation of ZN & MV = 0.360445344637529
Correlation of INDUS & MV = -0.48372517128143366
Correlation of CHAS & MV = 0.1752601777529185
Correlation of NOX & MV = -0.4273207763683772
Correlation of RM & MV = 0.6953599371272672
Correlation of AGE & MV = -0.3769545671428867
Correlation of DIS & MV = 0.24992873873512172
Correlation of RAD & MV = -0.38162623156691683
Correlation of TAX & MV = -0.46853593528654536
Correlation of PT & MV = -0.5077867038116086
Correlation of B & MV = 0.3334608226834165
Correlation of LSTAT & MV = -0.7376627294671615
Correlation of MV & MV = 1.0


### Build a linear regression model to predict house price

In [31]:
# Using Vector assembler to transform each row into a vector
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(df)
vhouse_df = vhouse_df.select(['features', 'MV'])
vhouse_df.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



In [32]:
# Spliting the dataset into training and testing datasets
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [34]:
# Building a Linear Regression Model
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
# fitting the Linear Regression Model
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.052058153125731824,0.015160702613403361,-0.0022856544276884296,2.3304123860866874,-9.084689346704783,4.875328751573695,0.0,-0.7138562453800125,0.0,0.0,-0.7287742327610004,0.008863091711978015,-0.475254299463443]
Intercept: 16.08220721746163


### Evaluate the Linear Regression model by getting the RMSE and R-squared values


In [35]:
# Printing the RMSE & R2 Values of the model
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.954361
r2: 0.729889


In [36]:
# Comparing the Predicted and Visually
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 39.40886548852506|       50.0|[0.01381,80.0,0.4...|
|26.718045209135067|       33.0|[0.019509999,17.5...|
|21.296230792989718|20.10000038|[0.019649999,80.0...|
|31.780456334234508|31.10000038|[0.02187,60.0,2.9...|
|29.827737626408847|31.20000076|[0.03049,55.0,3.7...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.667449


In [37]:
# Print RMSE for Test Data
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4.6775


In [38]:
# Printing Residuals
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.5, 0.4312430719370162, 0.23160069435274414, 0.20875888867323367, 0.1776149405452768, 0.174292059247129, 0.17346225737163024, 0.1727680460535628, 0.1719681969014127, 0.17133318021467492, 0.17124056704309146]


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
| -6.598290419802584|
| 0.4370511636233374|
| -6.057374402630835|
|0.23507550456013604|
|  4.306663367486223|
| 0.8277298230094274|
| -0.148800660600795|
|-1.8864481842497405|
| -3.458240947044267|
|  7.350843249974005|
|   7.96581711383044|
|    3.3592265714117|
| 0.8188559012497336|
| -3.448206832938766|
|  9.370192948479037|
|-1.6717296610909074|
| 4.8043120997907565|
|-10.039493509124977|
| -4.441407908596759|
| 2.8184193731676928|
+-------------------+
only showing top 20 rows



In [39]:
# Checking the predictions
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 39.40886548852506|       50.0|[0.01381,80.0,0.4...|
|26.718045209135067|       33.0|[0.019509999,17.5...|
|21.296230792989718|20.10000038|[0.019649999,80.0...|
|31.780456334234508|31.10000038|[0.02187,60.0,2.9...|
|29.827737626408847|31.20000076|[0.03049,55.0,3.7...|
|20.367368222757325|       17.5|[0.031129999,0.0,...|
| 30.17707909321033|33.40000153|[0.032370001,0.0,...|
| 32.94006729351027|34.90000153|[0.03359,75.0,2.9...|
| 28.36818296962332|24.10000038|[0.034449998,82.5...|
| 22.94367844861062|20.89999962|[0.03548,80.0,3.6...|
|24.835426248519568|22.89999962|[0.03551,25.0,4.8...|
| 26.25895063781101|23.20000076|[0.038710002,52.5...|
| 27.26792371228047|       22.0|[0.03932,0.0,3.41...|
|21.539118419160772|21.10000038|[0.039609998,0.0,...|
| 28.67586727262313|       28.0|[0.041129999,25.0...|
|27.404032032603443|22.89999